In [15]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [16]:
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2", 
    task="conversational"
)
model = ChatHuggingFace(llm=llm)

In [17]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [23]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = model.invoke(prompt).content

    return {'joke': response}

In [24]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = model.invoke(prompt).content

    return {'explanation': response}

In [25]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [26]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': " Why did the tomato turn red when it saw the pizza being made?\n\nBecause it saw the sauce!\n\nBut seriously, who doesn't love a good pizza joke? Whether it's about the sauce, the toppings, or the infamous pizza face, pizza is always a delicious source of inspiration for a laugh. Just remember, the real joke is in every savory bite!",
 'explanation': ' This joke is a play on words and involves a bit of wordplay and association with common knowledge.\n\nTomatoes are naturally green when they are growing on the vine. As they ripen, they turn red. In the joke, the tomato is personified and given the ability to see. When the tomato "sees" the pizza being made, it turns red. The punchline is "because it saw the sauce!"\n\nThe joke relies on the double meaning of "saw." Normally, we think of seeing as something the eyes do. But in this context, "saw" can also refer to pizza sauce. The tomato is not seeing the sauce with its eyes, but rather the tomato is being tr

In [27]:

list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': " Why did the tomato turn red when it saw the pizza being made?\n\nBecause it saw the sauce!\n\nBut seriously, who doesn't love a good pizza joke? Whether it's about the sauce, the toppings, or the infamous pizza face, pizza is always a delicious source of inspiration for a laugh. Just remember, the real joke is in every savory bite!", 'explanation': ' This joke is a play on words and involves a bit of wordplay and association with common knowledge.\n\nTomatoes are naturally green when they are growing on the vine. As they ripen, they turn red. In the joke, the tomato is personified and given the ability to see. When the tomato "sees" the pizza being made, it turns red. The punchline is "because it saw the sauce!"\n\nThe joke relies on the double meaning of "saw." Normally, we think of seeing as something the eyes do. But in this context, "saw" can also refer to pizza sauce. The tomato is not seeing the sauce with its eyes, but rather th

In [28]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': " Why don't some pastas ever fight? Because they pasta each other's differences! (I know, I know, it's a cheesy joke!) But seriously, who can resist a good plate of pasta, am I right? Spaghetti or fettuccine, lasagna or ravioli - they're all just so delicious and harmonious in their own ways. So let's put aside our differences and twirl a strand of pasta together in celebration of this fantastic pantry staple! Or maybe that's just my pasta-tive attempt at humor... 😄",
 'explanation': ' This joke plays on the double meaning of the word "pasta," which refers both to the food we enjoy in various shapes and sizes, as well as a way to describe someone or something that is not a problem or an issue. The punchline "Because they pasta each other\'s differences!" cleverly combines these meanings to create a humorous play on words.\n\nThe explanation for the joke also emphasizes the universally appreciated nature of pasta. Whether it\'s spaghetti, fettuccine, lasagna,

In [29]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': " Why did the tomato turn red when it saw the pizza being made?\n\nBecause it saw the sauce!\n\nBut seriously, who doesn't love a good pizza joke? Whether it's about the sauce, the toppings, or the infamous pizza face, pizza is always a delicious source of inspiration for a laugh. Just remember, the real joke is in every savory bite!", 'explanation': ' This joke is a play on words and involves a bit of wordplay and association with common knowledge.\n\nTomatoes are naturally green when they are growing on the vine. As they ripen, they turn red. In the joke, the tomato is personified and given the ability to see. When the tomato "sees" the pizza being made, it turns red. The punchline is "because it saw the sauce!"\n\nThe joke relies on the double meaning of "saw." Normally, we think of seeing as something the eyes do. But in this context, "saw" can also refer to pizza sauce. The tomato is not seeing the sauce with its eyes, but rather the

In [30]:
list(workflow.get_state_history(config1))


[StateSnapshot(values={'topic': 'pizza', 'joke': " Why did the tomato turn red when it saw the pizza being made?\n\nBecause it saw the sauce!\n\nBut seriously, who doesn't love a good pizza joke? Whether it's about the sauce, the toppings, or the infamous pizza face, pizza is always a delicious source of inspiration for a laugh. Just remember, the real joke is in every savory bite!", 'explanation': ' This joke is a play on words and involves a bit of wordplay and association with common knowledge.\n\nTomatoes are naturally green when they are growing on the vine. As they ripen, they turn red. In the joke, the tomato is personified and given the ability to see. When the tomato "sees" the pizza being made, it turns red. The punchline is "because it saw the sauce!"\n\nThe joke relies on the double meaning of "saw." Normally, we think of seeing as something the eyes do. But in this context, "saw" can also refer to pizza sauce. The tomato is not seeing the sauce with its eyes, but rather th

# **Time Travel**

In [32]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

# **Fault Tolerance**

In [34]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [35]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str
    step3: str

In [36]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}    

In [37]:

# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [38]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
❌ Kernel manually interrupted (crash simulated).


In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)

In [41]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done'}, next=('step_2',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f07cfd9-9905-6abe-8001-7498f2de27d7'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-08-19T13:08:29.342991+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f07cfd9-9903-6ebc-8000-30ce85a13351'}}, tasks=(PregelTask(id='8fc94a59-7552-78e1-ebb8-3eb151e0206b', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'input': 'start'}, next=('step_1',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f07cfd9-9903-6ebc-8000-30ce85a13351'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-08-19T13:08:29.342272+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'ch